# Part 3A — MNIST CNN with Keras (tf.keras)
_Last updated: 2025-11-16_



This notebook implements a compact **CNN** in **Keras** (the high‑level API that ships with TensorFlow).
It reports training curves and test metrics, and saves a small JSON for the comparison notebook.


## Environment
Use pip (Colab) or conda (local). The code detects CPU/GPU automatically.

In [ ]:

import os, sys, time, json, pathlib
import numpy as np
import matplotlib.pyplot as plt
# Keras is part of TensorFlow
import tensorflow as tf
from tensorflow import keras
print("TensorFlow:", tf.__version__)
print("Keras API:", keras.__version__)
print("Devices:", tf.config.list_physical_devices())



## Dataset & preprocessing

We use **MNIST** (28×28 grayscale digits, 10 classes). Input is normalized to `[0, 1]`
and reshaped to `(N, 28, 28, 1)` for CNNs. Labels are integers `0..9`.


In [ ]:

from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32")/255.0
x_test  = x_test.astype("float32")/255.0
x_train = np.expand_dims(x_train, -1)  # (N,28,28,1)
x_test  = np.expand_dims(x_test, -1)

print("Train:", x_train.shape, y_train.shape, " Test:", x_test.shape, y_test.shape)


## Model
Simple ConvNet: `Conv(32)->Conv(64)->MaxPool->Dropout->Flatten->Dense(128)->Dense(10)`

In [ ]:

from tensorflow.keras import layers, models

def build_model():
    inputs = keras.Input(shape=(28,28,1))
    x = layers.Conv2D(32, 3, activation="relu")(inputs)
    x = layers.Conv2D(64, 3, activation="relu")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.25)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    outputs = layers.Dense(10, activation="softmax")(x)
    model = models.Model(inputs, outputs)
    return model

model = build_model()
model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.summary()


## Train

In [ ]:

import time
callbacks=[keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
t0=time.perf_counter()
hist = model.fit(x_train, y_train, validation_split=0.1, epochs=5, batch_size=128, callbacks=callbacks, verbose=1)
train_time = time.perf_counter()-t0
print("Train time (s):", train_time)


## Curves

In [ ]:

plt.figure(figsize=(6,4))
plt.plot(hist.history["loss"], label="train_loss")
plt.plot(hist.history["val_loss"], label="val_loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.grid(True); plt.title("Keras: loss")
plt.tight_layout()
plt.savefig("artifacts/keras_loss.png", dpi=150)
plt.show()

plt.figure(figsize=(6,4))
plt.plot(hist.history["accuracy"], label="train_acc")
plt.plot(hist.history["val_accuracy"], label="val_acc")
plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend(); plt.grid(True); plt.title("Keras: accuracy")
plt.tight_layout()
plt.savefig("artifacts/keras_acc.png", dpi=150)
plt.show()


## Evaluate & save metrics

In [ ]:

# Utility: save artifacts (plots & metrics)
import os, json, pathlib, time
ART = pathlib.Path("artifacts")
ART.mkdir(exist_ok=True)

def save_metrics(name, **metrics):
    path = ART / f"{name}_metrics.json"
    with open(path, "w") as f:
        json.dump(metrics, f, indent=2)
    print("Saved:", path)

def effective_loc(*funcs):
    import inspect
    n = 0
    for f in funcs:
        try:
            src = inspect.getsource(f)
            for line in src.splitlines():
                s = line.strip()
                if s and not s.startswith("#"):
                    n += 1
        except Exception:
            pass
    return n

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
params = model.count_params()
save_metrics("keras", framework="keras", test_accuracy=float(test_acc), test_loss=float(test_loss),
             train_time_sec=float(train_time), params=int(params), epochs=int(len(hist.history["loss"])),
             device=str(tf.config.list_physical_devices("GPU") or tf.config.list_physical_devices("CPU")),
             effective_loc=int(effective_loc(build_model)))
model.save("artifacts/keras_mnist.h5")
print("Saved model and metrics to artifacts/")
